In [1]:
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser 
from langchain.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [7]:
model.invoke("Erkläre LLMs in drei Sätzen")
# returns JSON data with model output and metadata 

AIMessage(content='LLMs (Large Language Models) sind leistungsstarke künstliche Intelligenzmodelle, die auf großen Mengen an Textdaten trainiert werden, um natürliche Sprache zu verstehen und zu generieren. Sie können komplexe Aufgaben wie Übersetzung, Zusammenfassung und Antwortgenerierung durchführen und haben eine breite Palette von Anwendungen in der Sprachverarbeitung und anderen Bereichen. LLMs wie GPT-3 von OpenAI oder BERT von Google haben das Potenzial, die Art und Weise zu verändern, wie wir mit Computern interagieren und neue Möglichkeiten für die Automatisierung von Aufgaben zu schaffen.', response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 18, 'total_tokens': 169}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None}, id='run-84e6034a-1ac6-4017-ba5a-e5765b6a05b4-0')

In [24]:
response = model.invoke("Erkläre LLMs in drei Sätzen")
response.content

'LLM steht für Master of Laws und ist ein postgraduales Studium für Juristen, die ihre Kenntnisse in einem bestimmten Rechtsgebiet vertiefen möchten. Es wird in der Regel nach dem Abschluss eines Jurastudiums absolviert und dauert meist ein Jahr. Durch ein LLM-Programm können Juristen ihre Karrierechancen verbessern und sich auf spezifische berufliche Herausforderungen vorbereiten.'

In [3]:
parser = StrOutputParser() # Initialisierung eines Objekts zum Extrahieren des Strings 
chain = model | parser # Output des Model wird in parser Objekt gepiped
chain.invoke("Erkläre LLMs in drei Sätzen")

'LLMs sind Large Language Models, die auf künstlicher Intelligenz basieren und komplexe Sprachmuster analysieren können. Sie werden häufig für Textgenerierung, Übersetzung und andere sprachbezogene Aufgaben eingesetzt. LLMs wie GPT-3 von OpenAI sind bekannt für ihre Fähigkeit, menschenähnliche Texte zu erstellen und haben das Potenzial, viele Bereiche des menschlichen Lebens zu beeinflussen.'

In [27]:
template = """
Beantworte die Frage basierend auf dem unten gegebenen Kontext. 
Wenn das Beantworten der Frage nicht möglich ist durch den gegebenen Kontekt, antworte "Ich weiß es nicht". 

Kontext: {context}

Frage: {question}
"""

prompt = ChatPromptTemplate.from_template(template) # Template Prompt Objekt erzeugen


In [45]:
model.invoke(prompt.format(context="Es gibt auschließlich weiße Schwäne.", question = "Gibt es schwarze Schwäne?")).content


'Ich weiß es nicht.'

In [46]:
chain = prompt | model | parser
chain.invoke({
    "context": "Es gibt ausschließlich weiße Schwäne.", 
    "question": "Gibt es schwarze Schwäne?"
})

'Basierend auf dem gegebenen Kontext gibt es keine schwarzen Schwäne.'

In [66]:
from operator import itemgetter

diff_prompt = ChatPromptTemplate.from_template(
    "Übersetze {answer} in {language}"
)

diff_chain = (
    # dic um Input zu definieren, dies wird dann in diff_prompt gepiped
    {"answer": chain, # Antwort wird aus der anderen Chain als Input genommen
     "language": itemgetter("language")} # Sprache aus Dictonary
    | diff_prompt | model | parser 
)

diff_chain.invoke(
    {
        "context": "Alle Schwäne sind weiß, außer in Australien. Dort gibt es schwarze.",
        "question": "Gibt es schwarze Schwäne?", 
        "language": "Italienisch"
    }
)

'Sì, in Australia ci sono cigni neri.'

In [4]:
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Gibt es schwarze Schwäne?")

print(f"Embedding length: {len(embedded_query)}") # Dimension der Embedding Liste

Embedding length: 1536
[0.00901121263534296, -0.03669295798076368, -0.00873078020641617, -0.025039455366582045, -0.01990443465229751, 0.006879930087054519, -0.0007376917735520665, -0.012351468412199479, -0.009098458010849193, -0.04162856202435205]


In [6]:
sen1 = embeddings.embed_query("Es gibt in Europa nur weiße Schwäne")
sen2 = embeddings.embed_query("Es gibt schwarze Schwäne in Australien")

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

query_sen1_sim = cosine_similarity([embedded_query], [sen1])[0][0]
query_sen2_sim = cosine_similarity([embedded_query], [sen2])[0][0]
query_sen1_sim, query_sen2_sim

(0.8740940461805541, 0.9229002354973087)